In [ ]:
import numpy as np
import cv2
import rosbag
from cv_bridge import CvBridge

# Standard plotly imports
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

In [ ]:
# Load depth map    
bag = rosbag.Bag("/home/user/rosbags/2021-12-03-20-27-23.bag")

depth_imgs = []
imgs = []
i = 0
for topic, msg, t in bag.read_messages(topics="/zed2i/zed_node/depth/depth_registered"):
    if i % 30 == 0:
        depth_imgs.append(msg)
    i += 1
for topic, msg, t in bag.read_messages(topics="/zed2i/zed_node/left/image_rect_color"):
    if i % 30 == 0:
        # Convert from image msg to numpy array
        img = CvBridge().imgmsg_to_cv2(msg, desired_encoding="rgb8")
        imgs.append(img)
    i += 1
bag.close()

Extract one 2D image and depth image

In [ ]:
# 2D image
img = imgs[7]
px.imshow(img).show()

# Depth image
depth_img = CvBridge().imgmsg_to_cv2(depth_imgs[8], desired_encoding="32FC1")
# Replace points without range information
# depth_img_no_nans = np.nan_to_num(depth_img, 40.0)
px.imshow(depth_img)

Using the trained network, detectron2 outputs the following box

``array([[522.8492 , 440.39346, 724.945  , 546.60345]], dtype=float32)``

This box can now be cut from the image to get the ramp region

In [ ]:
box = np.array([522.8492 , 440.39346, 724.945  , 546.60345], dtype=np.float32)

# Test 2D image first
crop_img = img.copy()
crop_img = crop_img[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
px.imshow(crop_img).show()

# Now the same for the 3D image
crop_depth_img = depth_img.copy()
crop_depth_img = crop_depth_img[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
px.imshow(crop_depth_img)